In [ ]:
# Check if we're on Google Colab to clone and change dir into the repo
if 'google.colab' in str(get_ipython()):
  !git clone https://github.com/kk-digital/kcg-ml-sd1p4
  %cd kcg-ml-sd1p4

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python3 ./download_models.py

In [ ]:
!python3 ./process_models.py

In [ ]:
import os
import sys
import torch
import time
import shutil
from torchvision.transforms import ToPILImage
from os.path import join

base_directory = "../"
sys.path.insert(0, base_directory)

from stable_diffusion.constants import IODirectoryTree
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_image import *
from stable_diffusion.utils_model import *
from utility.utils_logger import *
from stable_diffusion.stable_diffusion import StableDiffusion
from utility.labml import monit


output_base_dir = join(base_directory, "./output/sd2-notebook/")
output_directory = join(output_base_dir, "denoising/")


try:
    shutil.rmtree(output_directory)
except Exception as e:
    print(e, "\n", "Creating the path...")
    os.makedirs(output_directory, exist_ok=True)
else:
    os.makedirs(output_directory, exist_ok=True)


def to_pil(image):
    return ToPILImage()(torch.clamp((image + 1.0) / 2.0, min=0.0, max=1.0))

In [ ]:
device = get_device()
base_dir = os.getcwd()
sys.path.insert(0, base_dir)

import configparser
config = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read(os.path.join(base_dir, "config.ini"))
config['BASE']['BASE_DIRECTORY'] = base_dir
config["BASE"].get('base_io_directory')

batch_size = 1
pt = IODirectoryTree(base_io_directory_prefix = config["BASE"].get('base_io_directory_prefix'), base_directory=base_dir)
pt.create_directory_tree_folders()

In [ ]:
# initialize an empty stable diffusion class
stable_diffusion = StableDiffusion(device=device)
get_memory_status()

In [ ]:
# initialize an empty latent diffusion model; it returns self.model
# then load the clip text embedder from the path `pt.embedder_path` with .load_clip_embedder()
# it returns the clip embedder, so you can chain a .load_submodels() to load the text embedder submodels

stable_diffusion.quick_initialize().load_clip_embedder(**pt.embedder).load_submodels(
    **pt.embedder_submodels
)
get_memory_status()

In [ ]:
stable_diffusion.model.clip_embedder

In [6]:
# get the embedding for a prompt
prompt_embedding = stable_diffusion.model.clip_embedder(
    ["A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta"]
)
null_prompt = stable_diffusion.model.clip_embedder([""])

In [ ]:
get_memory_status()
prompt_embedding.shape, null_prompt.shape

In [ ]:
# we don't need the embedder anymore, so we can unload it
stable_diffusion.model.unload_clip_embedder()
get_memory_status()

In [9]:
# let's save the prompt embedding
torch.save(prompt_embedding, join(output_directory, "prompt_embedding.pt"))
torch.save(null_prompt, join(output_directory, "null_prompt.pt"))

In [ ]:
# the latent diffusion class has a method to load the unet, since it is a submodel of it. it returns the unet model, wrapped in a DiffusionWrapper class.
# it is accessible as self.model.model or through the alias self.model.unet
stable_diffusion.model.load_unet(**pt.unet)
get_memory_status()

In [ ]:
# sample a latent representation, we know beforehand that the latent space is shaped as (1, 4, 64, 64)
initial_latent = torch.randn(1, 4, 64, 64, device=device)
get_memory_status()

In [ ]:
# define a timestep for this sample
time_step = torch.tensor([15.0]).to(device)
time_step.shape

In [13]:
def sample(
    stable_diffusion: StableDiffusion,
    shape: List[int],
    cond: torch.Tensor,
    repeat_noise: bool = False,
    temperature: float = 1.0,
    x_last: Optional[torch.Tensor] = None,
    uncond_scale: float = 7.5,
    uncond_cond: Optional[torch.Tensor] = None,
    skip_steps: int = 0,
    noise_fn=torch.randn,
):
    """
    ### Sampling Loop
    :param shape: is the shape of the generated images in the
        form `[batch_size, channels, height, width]`
    :param cond: is the conditional embeddings $c$
    :param temperature: is the noise temperature (random noise gets multiplied by this)
    :param x_last: is $x_{\tau_S}$. If not provided random noise will be used.
    :param uncond_scale: is the unconditional guidance scale $s$. This is used for
        $\epsilon_\theta(x_t, c) = s\epsilon_\text{cond}(x_t, c) + (s - 1)\epsilon_\text{cond}(x_t, c_u)$
    :param uncond_cond: is the conditional embedding for empty prompt $c_u$
    :param skip_steps: is the number of time steps to skip $i'$. We start sampling from $S - i'$.
        And `x_last` is then $x_{\tau_{S - i'}}$.
    """
    # Get device and batch size
    set_seed(0)
    latents = []
    device = stable_diffusion.sampler.model.device
    bs = shape[0]
    # Get $x_{\tau_S}$
    x = x_last if x_last is not None else noise_fn(shape, device=device)
    latents.append(x)
    # Time steps to sample at $\tau_{S - i'}, \tau_{S - i' - 1}, \dots, \tau_1$
    time_steps = np.flip(stable_diffusion.sampler.time_steps)[skip_steps:]
    for i, step in monit.enum("Sample", time_steps):
        # Index $i$ in the list $[\tau_1, \tau_2, \dots, \tau_S]$
        index = len(time_steps) - i - 1
        # Time step $\tau_i$
        ts = x.new_full((bs,), step, dtype=torch.long)
        # Sample $x_{\tau_{i-1}}$
        x, pred_x0, e_t = stable_diffusion.sampler.p_sample(
            x,
            cond,
            ts,
            step,
            index=index,
            repeat_noise=repeat_noise,
            temperature=temperature,
            uncond_scale=uncond_scale,
            uncond_cond=uncond_cond,
            noise_fn=noise_fn,
        )
        latents.append(x)
    # Return $x_0$
    return latents

In [ ]:
autocast = get_autocast()
with autocast:
    with torch.no_grad():
        latents = sample(
            stable_diffusion=stable_diffusion,
            shape=[1, 4, 64, 64],
            cond=prompt_embedding,
            uncond_cond=null_prompt,
            x_last=initial_latent,
            noise_fn=torch.randn,
        )
get_memory_status()

In [ ]:
grid = torch.cat(latents[1:], dim=0)
grid.shape

In [ ]:
grid = torchvision.utils.make_grid(
    grid,
    nrow=10,
    normalize=False,
    range=(-1, 1),
    scale_each=True,
    pad_value=0,
)
dim_grid_image = to_pil(grid)
dim_grid_image.save(join(output_directory, f"denoising_process.png"))
dim_grid_image

In [ ]:
stable_diffusion.model.unload_submodels()
torch.cuda.empty_cache()
get_memory_status()

In [18]:
to_be_decoded = latents[-1]

In [ ]:
stable_diffusion.model.load_autoencoder(**pt.autoencoder).load_decoder(**pt.decoder)
get_memory_status()

In [20]:
decoded_image = stable_diffusion.decode(to_be_decoded)

In [ ]:
to_pil(decoded_image.squeeze())

In [22]:
%%capture
stable_diffusion.model.load_unet(**pt.unet)

In [ ]:
images = stable_diffusion.generate_images_from_embeddings(
    embedded_prompt=prompt_embedding, null_prompt=null_prompt
)

In [ ]:
print(images.shape)
to_pil(images.squeeze())